In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
FLDR='drive/MyDrive/dogs-vs-cats/'


In [3]:
import os

In [4]:
files_zip_full=os.listdir(FLDR)

In [5]:
files_zip_full

['sampleSubmission.csv', 'test1.zip', 'train.zip', 'weights.h5']

In [6]:
train_data=files_zip_full[2]

In [7]:
train_data

'train.zip'

In [8]:
import zipfile
import cv2

In [10]:
archive=zipfile.ZipFile(FLDR+train_data)
archive.extractall(FLDR+'extracted')
  

In [11]:
files_zip_full=os.listdir(FLDR)

In [12]:
files_zip_full

['sampleSubmission.csv', 'test1.zip', 'train.zip', 'weights.h5', 'extracted']

In [13]:
test_data=files_zip_full[1]

In [14]:
archive=zipfile.ZipFile(FLDR+test_data)
archive.extractall(FLDR+'extracted_2')

In [15]:
files_zip_full=os.listdir(FLDR)

In [16]:
files_zip_full

['sampleSubmission.csv',
 'test1.zip',
 'train.zip',
 'weights.h5',
 'extracted',
 'extracted_2']

In [19]:
training_data=files_zip_full[4]+'/train/'

In [20]:
training_data

'extracted/train/'

In [ ]:
Labels=[]
Images=[]

for f_n in os.listdir(FLDR+training_data):
  print(f_n)
  if 'cat' in f_n or 'Cat' in f_n:
    img=cv2.imread(FLDR+training_data+f_n,cv2.IMREAD_UNCHANGED)
    img=cv2.resize(img,(224,224))
    Images.append(img)
    Labels.append(0)
  if 'dog' in f_n or 'Dog' in f_n:
    img=cv2.imread(FLDR+training_data+f_n,cv2.IMREAD_UNCHANGED)
    img=cv2.resize(img,(224,224))
    Images.append(img)
    Labels.append(1)

In [35]:
import numpy as np

In [44]:
from sklearn.model_selection import train_test_split
X_train,X_val,Y_train,Y_val= train_test_split(np.array(Images),np.array(Labels),test_size=0.15)

In [55]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from keras import optimizers
from keras.callbacks import ModelCheckpoint

In [58]:
def model(shape=(224,224,3)):
  input_tensor=Input(shape=shape)             

  base_model = ResNet50V2(weights="imagenet", include_top=False, input_tensor=input_tensor)   

  model = Sequential([
        base_model,
        Flatten(), 
        Dense(512, activation='relu'),
        Dropout(0.3),         
        Dense(1, activation='sigmoid')    
    ])

  for layer in model.layers:
    layer.trainable = True
  model.compile(optimizer=optimizers.Adam(lr=0.01), loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [59]:
Model=model()

In [60]:
Model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
flatten_4 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               51380736  
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 513       
Total params: 74,946,049
Trainable params: 74,900,609
Non-trainable params: 45,440
_________________________________________________________________


In [ ]:
checkpoint = ModelCheckpoint(FLDR+"Model.h5", monitor='val_accuracy', save_best_only=True, mode='max') #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]
history = Model.fit(X_train,Y_train,validation_data =(X_val,Y_val),callbacks=callbacks_list,verbose=1,epochs=30,batch_size=32)